In [ ]:
from sklearn                         import datasets
from pybrain.datasets.classification import ClassificationDataSet
from pybrain.supervised.trainers     import BackpropTrainer
from pybrain.tools.shortcuts         import buildNetwork
import matplotlib.pyplot as plt

iris = datasets.load_iris()
x, y = iris.data, iris.target

dataset = ClassificationDataSet(4,1,nb_classes=3)

for i in range(len(x)):
    dataset.addSample(x[i], y[i])

train_data, part_data = dataset.splitWithProportion(0.6)
print("quantidade de treino: %d" % len(train_data))

test_data, val_data = part_data.splitWithProportion(0.5)
print("quantidade de teste: %d" % len(test_data))
print("quantidade de validação: %d" % len(val_data))

net = buildNetwork(dataset.indim, 3, dataset.outdim)
trainer = BackpropTrainer(net, dataset=train_data, learningrate=0.01,momentum=0.1, verbose=True)
train_errors, val_errors = trainer.trainUntilConvergence(dataset=train_data, maxEpochs=100)

plt.plot(train_errors, 'b', val_errors, 'r')
plt.show()

In [2]:
from sklearn                         import datasets
from pybrain.datasets.classification import ClassificationDataSet
from pybrain.supervised.trainers     import BackpropTrainer
from pybrain.tools.shortcuts         import buildNetwork
from pybrain.structure.modules       import SoftmaxLayer
from pybrain.utilities               import percentError
import matplotlib.pyplot as plt
import numpy as np


iris = datasets.load_iris()
x, y = iris.data, iris.target
dataset = ClassificationDataSet(4,1, nb_classes=3)

for i in range(len(x)):
    dataset.addSample(x[i], y[i])

train_data_temp, part_data_temp = dataset.splitWithProportion(0.6)
test_data_temp, val_data_temp = part_data_temp.splitWithProportion(0.5)

train_data = ClassificationDataSet(4,1,nb_classes=3)
for n in range(train_data_temp.getLength()):
    train_data.addSample(train_data_temp.getSample(n)[0], train_data_temp.getSample(n)[1])
    
test_data = ClassificationDataSet(4,1,nb_classes=3)
for n in range(test_data_temp.getLength()):
    test_data.addSample(test_data_temp.getSample(n)[0], test_data_temp.getSample(n)[1])

val_data = ClassificationDataSet(4,1,nb_classes=3)
for n in range(val_data_temp.getLength()):
    val_data.addSample(val_data_temp.getSample(n)[0], val_data_temp.getSample(n)[1])

print('Before _convertToOneOfMany...')
print(train_data['target'][:5])

train_data._convertToOneOfMany()
test_data._convertToOneOfMany()
val_data._convertToOneOfMany()

print('After _convertToOneOfMany...')
print(train_data['target'][:5])

print('--------------------------------------')

net = buildNetwork(4,5,3, outclass=SoftmaxLayer)
trainer = BackpropTrainer(net, dataset=train_data, learningrate=0.01, momentum=0.1)
trainer.trainOnDataset(train_data, 100)

out = net.activateOnDataset(test_data).argmax(axis=1)
print("Erro de teste: %f" % percentError(out, test_data['class']))

print('--------------------------------------')

out = net.activateOnDataset(val_data).argmax(axis=1)
print("Erro de validação: %f" % percentError(out, val_data['class']))

print('--------------------------------------')

print("Validação")
print('saida da rede:\t', out)
print('correto:\t', val_data['class'][:,0])



Before _convertToOneOfMany...
[[1]
 [0]
 [1]
 [2]
 [0]]
After _convertToOneOfMany...
[[0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]]
--------------------------------------
Erro de teste: 6.666667
--------------------------------------
Erro de validação: 3.333333
--------------------------------------
Validação
saida da rede:	 [2 1 2 2 2 2 0 1 0 1 1 2 1 2 2 1 0 0 1 2 2 2 1 0 0 1 2 2 0 2]
correto:	 [2 1 2 2 2 2 0 1 0 1 1 2 1 2 2 1 0 0 1 2 2 2 1 0 0 1 2 1 0 2]
